In [1]:
from google.colab import drive

drive.mount('/content/drive')
file_path = '/content/drive/My Drive/hot_or_not.csv'

import pandas as pd

df = pd.read_csv(file_path)
df.head()

Mounted at /content/drive


,name,is_hot,genre,style,popularity,engagement,valence,arousal,danceable,not_danceable,...,spectral_contrast_7,spectral_rolloff,tempo,zero_crossing_rate,tonnetz_1,tonnetz_2,tonnetz_3,tonnetz_4,tonnetz_5,tonnetz_6
0,Don't Start Now Dua Lipa,1,dance,dance,0.533699,0.806275,5.426029,4.416872,0.978744,0.021256,...,47.508168,5733.425143,123.046875,0.108134,0.135655,-0.035423,0.074641,-0.040866,-0.001042,-0.023227
1,"MAMIII Becky G, KAROL G",1,reggae,pop,0.697525,0.714739,5.250370,4.311930,0.884116,0.115884,...,46.515226,5100.492213,95.703125,0.108238,0.182400,-0.023066,0.050366,0.023565,0.020355,-0.000741
2,More Than A Woman Bee Gees,1,electronic,soul,0.775511,0.839151,5.416806,4.172360,0.978051,0.021949,...,47.952240,4811.615524,107.666016,0.098131,0.140643,-0.035769,0.103387,0.058251,-0.005618,0.007339
3,Calm Down Rema,1,latin,electronic,0.588689,0.666051,5.269323,4.302434,0.989999,0.010001,...,47.629641,4737.569885,143.554688,0.071252,-0.049012,-0.179290,0.180313,0.023707,-0.044212,-0.022441
4,"Ayo Girl (Fayahh Beat) [feat. Rema] Robinson, ...",1,hiphop,rnb,0.685297,0.711811,5.431922,4.295860,0.996658,0.003342,...,47.046475,6217.922652,129.199219,0.108013,-0.190897,-0.107207,0.015708,0.012103,-0.002555,-0.002424


In [2]:
!pip install xgboost catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 MB 6.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-runtime-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cudnn-cu12==8.9.2.26; platform_system == "Linux" an

In [11]:
import xgboost as xgb
from catboost import CatBoostClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.ensemble import RandomForestClassifier

# classification with Random Forest

In [82]:
df_encoded = pd.get_dummies(df, columns=['genre', 'style','mood_group'])

X = df_encoded.drop(['name','is_hot','not_danceable','not_aggressive','not_happy','not_sad','not_relaxing','not_acoustic','not_electronic','male','instrumental','dark'], axis=1)
y = df_encoded['is_hot']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

random_forest_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=20,
    random_state=42,
    verbose=1
)

random_forest_model.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    1.0s


RandomForestClassifier(max_depth=20, random_state=42, verbose=1)

In [83]:
y_pred_random = random_forest_model.predict(X_test)

accuracy_xgb = accuracy_score(y_test, y_pred_random)
print(f'XGBoost Accuracy: {accuracy_xgb:.2f}')

f1_xgb = f1_score(y_test, y_pred_random)
print(f'XGBoost F1 Score: {f1_xgb:.2f}')

print(classification_report(y_test, y_pred_random))

XGBoost Accuracy: 0.83
XGBoost F1 Score: 0.46
              precision    recall  f1-score   support

           0       0.84      0.98      0.90       386
           1       0.81      0.32      0.46       109

    accuracy                           0.83       495
   macro avg       0.83      0.65      0.68       495
weighted avg       0.83      0.83      0.80       495



[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


# classification with XGBoost

In [84]:
df_encoded = pd.get_dummies(df, columns=['genre', 'style','mood_group'])

X = df_encoded.drop(['name','is_hot','not_danceable','not_aggressive','not_happy','not_sad','not_relaxing','not_acoustic','not_electronic','male','instrumental','dark'], axis=1)
y = df_encoded['is_hot']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_xgb = xgb.XGBClassifier(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=3,
    random_state=42,
    scale_pos_weight=4
)
model_xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [85]:
y_pred_xgb = model_xgb.predict(X_test)

accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print(f'XGBoost Accuracy: {accuracy_xgb:.2f}')

f1_xgb = f1_score(y_test, y_pred_xgb)
print(f'XGBoost F1 Score: {f1_xgb:.2f}')

print(classification_report(y_test, y_pred_xgb))

XGBoost Accuracy: 0.85
XGBoost F1 Score: 0.63
              precision    recall  f1-score   support

           0       0.89      0.92      0.90       386
           1       0.67      0.60      0.63       109

    accuracy                           0.85       495
   macro avg       0.78      0.76      0.77       495
weighted avg       0.84      0.85      0.84       495



# classification with CatBoost

In [87]:
X = df.drop(['name','is_hot','not_danceable','not_aggressive','not_happy','not_sad','not_relaxing','not_acoustic','not_electronic','male','instrumental','dark'], axis=1)
y = df['is_hot']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


cat_features = ['genre', 'style','mood_group']

model_cat = CatBoostClassifier(
    iterations=2500,
    learning_rate=0.01,
    depth=3,
    verbose=500,  # Controls how much information CatBoost prints during training
)

model_cat.fit(X_train, y_train, cat_features=cat_features)

0:	learn: 0.6883031	total: 16.1ms	remaining: 40.3s
500:	learn: 0.3196651	total: 4.48s	remaining: 17.9s
1000:	learn: 0.2657647	total: 9.01s	remaining: 13.5s
1500:	learn: 0.2189226	total: 13.7s	remaining: 9.13s
2000:	learn: 0.1849499	total: 16.9s	remaining: 4.22s
2499:	learn: 0.1587243	total: 20.2s	remaining: 0us


In [88]:
# Get feature importance
feature_importances = model_cat.get_feature_importance()

# Combine feature names and their importances
feature_importance_list = list(zip(X.columns, feature_importances))

# Sort the list by importance in descending order
feature_importance_list.sort(key=lambda x: x[1], reverse=True)

# Print out the sorted feature importances
for feature_name, importance in feature_importance_list:
    print(f"Feature: {feature_name}, Importance: {importance}")

Feature: vocal, Importance: 10.648377946937956
Feature: mfcc_4, Importance: 6.208088217425424
Feature: aggressive, Importance: 4.9300769519188234
Feature: mfcc_3, Importance: 4.875715094468191
Feature: happy, Importance: 3.9224022694228764
Feature: mood_group, Importance: 3.860773420754513
Feature: genre, Importance: 3.6535355002171745
Feature: spectral_contrast_1, Importance: 3.6368263980375737
Feature: bright, Importance: 3.2534204330193552
Feature: engagement, Importance: 3.1403141542129758
Feature: female, Importance: 2.9593185455703823
Feature: spectral_contrast_6, Importance: 2.9143784449865184
Feature: popularity, Importance: 2.844092091342122
Feature: electronic, Importance: 2.560163050811614
Feature: spectral_contrast_4, Importance: 1.974820071813095
Feature: spectral_contrast_7, Importance: 1.9581085158630431
Feature: relaxing, Importance: 1.8486534618911494
Feature: mfcc_10, Importance: 1.8118131832300624
Feature: style, Importance: 1.7885901849718213
Feature: tonnetz_3, Imp

In [89]:
y_pred_cat = model_cat.predict(X_test)

# Calculate accuracy
accuracy_cat = accuracy_score(y_test, y_pred_cat)
print(f'CatBoost Accuracy: {accuracy_cat:.2f}')

# Calculate F1 score
f1_cat = f1_score(y_test, y_pred_cat)
print(f'CatBoost F1 Score: {f1_cat:.2f}')

# Detailed classification report
print(classification_report(y_test, y_pred_cat))

CatBoost Accuracy: 0.86
CatBoost F1 Score: 0.61
              precision    recall  f1-score   support

           0       0.87      0.96      0.91       386
           1       0.78      0.50      0.61       109

    accuracy                           0.86       495
   macro avg       0.83      0.73      0.76       495
weighted avg       0.85      0.86      0.85       495



# Ensemble

In [91]:
import numpy as np
# Combine the predictions
y_pred = np.zeros_like(y_pred_random)
# Weighted voting strategy: more weight to CatBoost for Class 1
for i in range(len(y_pred_random)):
    votes = y_pred_random[i] + y_pred_xgb[i] + y_pred_cat[i]
    if votes >= 1:  # If someone predicts 1
        y_pred[i] = 1
    else:
        y_pred[i] = 0

# Calculate accuracy
accuracy_cat = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy_cat:.2f}')

# Calculate F1 score
f1_cat = f1_score(y_test, y_pred)
print(f'F1 Score: {f1_cat:.2f}')

# Detailed classification report
print(classification_report(y_test, y_pred))

Accuracy: 0.85
F1 Score: 0.64
              precision    recall  f1-score   support

           0       0.90      0.91      0.90       386
           1       0.66      0.62      0.64       109

    accuracy                           0.85       495
   macro avg       0.78      0.77      0.77       495
weighted avg       0.84      0.85      0.84       495



# download the models

In [92]:
import joblib
joblib.dump(random_forest_model, 'random_forest_model.pkl')
joblib.dump(model_xgb, 'xgboost_model.pkl')
joblib.dump(model_cat, 'catboost_model.pkl')

['catboost_model.pkl']